In [1]:
import sys
 
sys.path.append('..')
from piece_v2 import piece_v2

piece_v2.start()

In [7]:
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments
from soundmining_tools.supercollider_client import *
from soundmining_tools.supercollider_client import SupercolliderClient
from soundmining_tools.modular.instrument import AddAction
from soundmining_tools.generative import *
from piece_v2 import *
from soundmining_tools.spectrum import make_fm_synthesis

piece_v2.reset()
piece_v2.synth_player.start()

static_control = piece_v2.instruments.static_control
sine_control = piece_v2.instruments.sine_control
perc_control = piece_v2.instruments.perc_control
line_control = piece_v2.instruments.line_control
signal_sum = piece_v2.instruments.signal_sum
signal_multiply = piece_v2.instruments.signal_multiply
three_block_control = piece_v2.instruments.three_block_control
four_block_control = piece_v2.instruments.four_block_control

#Harmonic
mod_ratio1 = 5/1
mod_ratio2 = 6/1
car_ratio = 7/1
fundamental = 55

#Dark
#mod_ratio1 = 13/8
#mod_ratio2 = 8/13
#car_ratio = 1
#fundamental = 110


fm_1 = make_fm_synthesis(fundamental * car_ratio, fundamental * mod_ratio1, 12)
fm_2 = make_fm_synthesis(fundamental * car_ratio, fundamental * mod_ratio2, 12)

display(fm_1)
display(fm_2)

class MyHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)    

    # mod1 triangle, mod2 saw. Very harsh
    # tri, sine, saw. Also very harsh
    # tri, sine, tri. Punch, gritty, dark.
    # sine, sine sine. More mellow. Spooky in high reg

    def variant3(self, start: float, pitch: float, amp: float) -> None:
        duration = random_range(13, 21)

        #mod_index1 = line_control(random_int_range(3, 7) * amp, random_int_range(3, 7) * amp)
        #mod_index2 = line_control(random_int_range(2, 5) * amp, random_int_range(2, 5) * amp)

        mod_index1 = sine_control(random_range(0, 1), random_range(0, 2))
        mod_index2 = three_block_control(
            levels=[random_range(0, 1), random_range(0, 2), random_range(0, 2), random_range(0, 1)], 
            times=[0.4, 0.2, 0.4], 
            curves=[0, 0, 0])


        mod_freq1 = static_control(pitch * mod_ratio1)                                     
        mod_amp1 = signal_multiply(mod_freq1, mod_index1).add_action(AddAction.TAIL_ACTION)
        mod1 = (
            piece_v2.synth_player.note()
            .sine(freq=mod_freq1, amp=mod_amp1)
            .audio_stack.pop()
        )
        
        mod_freq2 = static_control(pitch * mod_ratio2)           
        mod_amp2 = signal_multiply(mod_freq2, mod_index2).add_action(AddAction.TAIL_ACTION)
        fm_mod1 = signal_sum(mod_freq2, mod1).add_action(AddAction.TAIL_ACTION)
        mod2 = (
            piece_v2.synth_player.note()
            .sine(freq=fm_mod1, amp=mod_amp2)
            .audio_stack.pop()
        )

        car_freq = static_control(pitch * car_ratio)
        car_amp = three_block_control(levels=[0, random_range(0.50, 0.75), random_range(0.50, 0.75), 0], times=[0.4, 0.2, 0.4], curves=[0, 0, 0])
        fm_mod = signal_sum(car_freq, mod2).add_action(AddAction.TAIL_ACTION)        
        #pan = line_control(random_range(-0.99, 0.99), random_range(-0.99, 0.99))
        pan_start, pan_end = pan_line(1.0)
        pan = line_control(pan_start, pan_end)
        (
            piece_v2.synth_player.note()
                .sine(freq=fm_mod, amp=car_amp)                                
                .pan(pan)
                #.stereo_hall_reverb(static_control(1.0))
                .play(start, duration=duration)
        )

    def variant2(self, start: float, pitch: float, amp: float) -> None:
        duration = random_range(13, 21)

        #mod_index1 = line_control(random_int_range(3, 7) * amp, random_int_range(3, 7) * amp)
        #mod_index2 = line_control(random_int_range(2, 5) * amp, random_int_range(2, 5) * amp)

        mod_index1 = sine_control(random_range(0, 1), random_range(0, 2))
        mod_index2 = three_block_control(
            levels=[random_range(0, 1), random_range(0, 2), random_range(0, 2), random_range(0, 1)], 
            times=[0.4, 0.2, 0.4], 
            curves=[0, 0, 0])

        mod_freq1 = static_control(pitch * mod_ratio1)                
        mod_amp1 = signal_multiply(static_control(pitch * mod_ratio1), mod_index1).add_action(AddAction.TAIL_ACTION)
        mod1 = (
            piece_v2.synth_player.note()
            .sine(freq=mod_freq1, amp=mod_amp1)
            .audio_stack.pop()
        )
        
        mod_freq2 = static_control(pitch * mod_ratio2)                
        mod_amp2 = signal_multiply(static_control(pitch * mod_ratio2), mod_index2).add_action(AddAction.TAIL_ACTION)
        fm_mod1 = signal_sum(mod_freq2, mod1).add_action(AddAction.TAIL_ACTION)
        mod2 = (
            piece_v2.synth_player.note()
            .sine(freq=fm_mod1, amp=mod_amp2)
            .audio_stack.pop()
        )

        car_freq = static_control(pitch * car_ratio)
        car_amp = three_block_control(levels=[0, random_range(0.50, 0.75), random_range(0.50, 0.75), 0], times=[0.4, 0.2, 0.4], curves=[0, 0, 0])
        fm_mod = signal_sum(car_freq, mod2).add_action(AddAction.TAIL_ACTION)
        #pan = random_range(-0.5, 0.5)
        pan = line_control(random_range(-0.99, 0.99), random_range(-0.99, 0.99))
        (
            piece_v2.synth_player.note()
                .sine(freq=fm_mod, amp=car_amp)
                .pan(pan)
                .play(start, duration=duration)
        )

    def variant1(self, start: float, pitch: float, amp: float) -> None:
        mod_index1 = line_control(random_int_range(3, 7) * amp, random_int_range(3, 7) * amp)
        mod_index2 = line_control(random_int_range(2, 5) * amp, random_int_range(2, 5) * amp)

        mod_freq1 = static_control(pitch * mod_ratio1)                
        mod_amp1 = signal_multiply(static_control(pitch * mod_ratio1), mod_index1).add_action(AddAction.TAIL_ACTION)
        mod1 = (
            piece_v2.synth_player.note()
            .sine(freq=mod_freq1, amp=mod_amp1)
            .audio_stack.pop()
        )
        
        mod_freq2 = static_control(pitch * mod_ratio2)                
        mod_amp2 = signal_multiply(static_control(pitch * mod_ratio2), mod_index2).add_action(AddAction.TAIL_ACTION)
        fm_mod1 = signal_sum(mod_freq2, mod1).add_action(AddAction.TAIL_ACTION)
        mod2 = (
            piece_v2.synth_player.note()
            .sine(freq=fm_mod1, amp=mod_amp2)
            .audio_stack.pop()
        )

        car_freq = static_control(pitch * car_ratio)
        car_amp = sine_control(0, amp)
        fm_mod = signal_sum(car_freq, mod2).add_action(AddAction.TAIL_ACTION)
        pan = random_range(-0.5, 0.5)
        (
            piece_v2.synth_player.note()
                .sine(freq=fm_mod, amp=car_amp)
                .pan(static_control(pan))
                .play(start, 5)
        )

    def handle_note(self, patch_arguments: PatchArguments) -> None:
        
        match patch_arguments.octave:
            case 2:
                pitch = fm_1[patch_arguments.note][0]
            case 3: 
                pitch = fm_1[patch_arguments.note][1]
            case 4:
                pitch = fm_2[patch_arguments.note][0]
            case 5:
                pitch = fm_2[patch_arguments.note][1]
            case default: 
                pitch = fm_1[patch_arguments.note][0]
    
        self.variant3(patch_arguments.start, pitch, patch_arguments.amp)


my_handler = MyHandler(piece_v2.supercollider_client)
piece_v2.receiver.set_note_handler(my_handler)

[(385.0, 385.0),
 (660.0, 110.0),
 (935.0, 165.0),
 (1210.0, 440.0),
 (1485.0, 715.0),
 (1760.0, 990.0),
 (2035.0, 1265.0),
 (2310.0, 1540.0),
 (2585.0, 1815.0),
 (2860.0, 2090.0),
 (3135.0, 2365.0),
 (3410.0, 2640.0)]

[(385.0, 385.0),
 (715.0, 55.0),
 (1045.0, 275.0),
 (1375.0, 605.0),
 (1705.0, 935.0),
 (2035.0, 1265.0),
 (2365.0, 1595.0),
 (2695.0, 1925.0),
 (3025.0, 2255.0),
 (3355.0, 2585.0),
 (3685.0, 2915.0),
 (4015.0, 3245.0)]

In [8]:
piece_v2.stop()

In [ ]:
from soundmining_tools.spectrum import make_fm_synthesis

mod_ratio1 = 13/5
mod_ratio2 = 13/8
car_ratio = 13/21

print(mod_ratio1, mod_ratio2, car_ratio)
fundamental = 110

fm_1 = make_fm_synthesis(fundamental * car_ratio, fundamental * mod_ratio1, 10)
fm_2 = make_fm_synthesis(fundamental * car_ratio, fundamental * mod_ratio1, 10)

display(fm_1)
display(fm_2)
